In [1]:
import pandas as pd
import sqlite3

## 1. Create a connection to the database using the library sqlite3.

In [2]:
con = sqlite3.connect('data/checking-logs.sqlite')

## 2. Get the schema of the table test.

In [3]:
schema_test = pd.read_sql('PRAGMA table_info(test);', con)
schema_test

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


## 3. Get only the first 10 rows of the table test to check what the table looks like.

In [4]:
pd.read_sql('SELECT * FROM test LIMIT 10;', con)

,uid,labname,first_commit_ts,first_view_ts
0,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


## 4. Find among all the users the minimum value of the delta between the first commit of the user and the deadline of the corresponding lab using only one query.

In [5]:
schema_deadlines = pd.read_sql('PRAGMA table_info(deadlines);', con)
schema_deadlines

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,labs,TEXT,0,None,0
2,2,deadlines,INTEGER,0,None,0


In [6]:
pd.read_sql('SELECT * FROM deadlines;', con)

,index,labs,deadlines
0,0,laba04,1587945599
1,1,laba04s,1587945599
2,2,laba05,1588550399
3,4,laba06,1590364799
4,5,laba06s,1590364799
5,3,project1,1589673599


In [7]:
query_min = """
SELECT uid,
    MIN (CAST(strftime('%s', first_commit_ts) AS INT) - deadlines) / 3600 AS min_diff
FROM test t
JOIN deadlines dl on t.labname = dl.labs
WHERE t.labname != 'project1'
GROUP BY uid
ORDER by min_diff ASC
LIMIT 1
"""

In [8]:
df_min = pd.read_sql(query_min, con)
df_min

,uid,min_diff
0,user_30,-202


## 5. Do the same thing, but for the maximum, using only one query, the dataframe name is df_max.

In [30]:
query_max = """
SELECT uid,
    MAX (CAST(strftime('%s', first_commit_ts) AS INT) - deadlines) / 3600 AS max_diff
FROM test t
JOIN deadlines dl on t.labname = dl.labs
WHERE t.labname != 'project1'
GROUP BY uid
ORDER by max_diff DESC
LIMIT 1
"""

In [31]:
df_max = pd.read_sql(query_max, con)
df_max

,uid,max_diff
0,user_25,-2


## 6. Do the same thing but for the average, using only one query, this time your dataframe should not include the uid column, and the dataframe name is df_avg.

In [11]:
query_avg = """
SELECT uid,
    AVG((CAST(strftime('%s', first_commit_ts) AS INT) - deadlines) / 3600) AS avg_diff
FROM test t
JOIN deadlines dl on dl.labs = t.labname
WHERE t.labname != 'project1'
"""

In [12]:
df_avg = pd.read_sql(query_avg, con)
df_avg

,uid,avg_diff
0,user_30,-89.125


## 7. Hypothesis test.

In [13]:
query_hyp = """
SELECT t.uid,
    AVG((CAST(strftime('%s', t.first_commit_ts) AS INT) - dl.deadlines) / 3600) AS avg_diff,
    pv.pageviews
FROM test t
JOIN deadlines dl on dl.labs = t.labname
LEFT JOIN (
    SELECT uid,
    COUNT(*) AS pageviews
    FROM pageviews
    GROUP BY uid
) pv ON pv.uid = t.uid
WHERE t.labname != 'project1'
GROUP BY t.uid
"""
view_diff = pd.read_sql(query_hyp, con)
view_diff

,uid,avg_diff,pageviews
0,user_1,-64.400000,28
1,user_10,-74.800000,89
2,user_14,-159.000000,143
3,user_17,-61.600000,47
4,user_18,-5.666667,3
5,user_19,-98.750000,16
6,user_21,-95.500000,10
7,user_25,-92.600000,179
8,user_28,-86.400000,149
9,user_3,-105.400000,317


In [14]:
view_diff[['avg_diff', 'pageviews']].corr()

,avg_diff,pageviews
avg_diff,1.000000,-0.279736
pageviews,-0.279736,1.000000


In [15]:
con.close()